# Table 8 Reproduction v2: More Training

**Αλλαγές από v1:**
- `max_steps`: 1500 → **3000**
- `max_train_samples`: 3000 → **10000**
- `algo.var_min`: false → **true**

**Στόχος:** Να δούμε αν με περισσότερο training παίρνουμε τη σωστή διάταξη:
- L'=4: U[0.45,0.95] < U[0.3,0.8] < Linear

In [1]:
# Cell 1: Setup - Clone repo (αν δεν υπάρχει)
import os
if not os.path.exists('/content/bd3lms'):
    !cd /content && git clone https://github.com/ntua-el21050/bd3lms.git
else:
    print("Repo already exists, skipping clone")

# Create runs directory
!mkdir -p /content/repro_runs_v2

Cloning into 'bd3lms'...
remote: Enumerating objects: 768, done.
remote: Counting objects: 100% (226/226), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 768 (delta 202), reused 176 (delta 176), pack-reused 542 (from 1)
Receiving objects: 100% (768/768), 1.78 MiB | 35.67 MiB/s, done.
Resolving deltas: 100% (496/496), done.


In [2]:
# Cell 2: Install dependencies (αν χρειάζεται)
!pip install -q \
    torchmetrics==1.6.2 \
    datasets==3.3.2 \
    einops==0.8.1 \
    fsspec==2024.2.0 \
    hydra-core==1.3.2 \
    lightning==2.5.0.post0 \
    omegaconf==2.3.0 \
    packaging==23.2 \
    pandas==2.2.1 \
    rich==13.7.1 \
    scikit-learn==1.5.1 \
    timm==0.9.16 \
    transformers==4.49.0 \
    matplotlib==3.10.0 \
    wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 141.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 140.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# Cell 3: Helper functions
import subprocess
import re
import os
import shutil
import sys
from pathlib import Path

def run_main(overrides, timeout=None):
    """Run main.py with given overrides."""
    env = dict(os.environ)
    env.setdefault("HYDRA_FULL_ERROR", "1")
    cmd = [sys.executable, "-u", "bd3lms/main.py", *overrides]
    print("\n$", " ".join(cmd))
    proc = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        timeout=timeout,
        check=False,
        env=env,
    )
    print(proc.stdout[-4000:])
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed with return code {proc.returncode}")
    return proc.stdout

def extract_val_ppl(log_text: str):
    for line in reversed(log_text.splitlines()):
        if "val/ppl" in line.lower():
            m = re.search(r"val/ppl.*?([0-9]+(?:\.[0-9]+)?(?:e[+-]?\d+)?)", line, re.IGNORECASE)
            if m:
                return float(m.group(1))
    return None

def _small_loader_overrides(batch_size=8, num_workers=2):
    return [
        f"loader.global_batch_size={batch_size}",
        f"loader.eval_global_batch_size={batch_size}",
        f"loader.batch_size={batch_size}",
        f"loader.eval_batch_size={batch_size}",
        f"loader.num_workers={num_workers}",
        "trainer.accumulate_grad_batches=1",
    ]

def train_run_v2(
    run_name,
    algo,
    block_size=128,
    from_pretrained=None,
    max_steps=10000,        # Αύξησε από 3000
    max_train_samples=20000,  # Αύξησε από 10000
    extra_overrides=None,
):
    """Train a model with MORE training."""
    save_dir = Path("/content/repro_runs_v2") / run_name
    if save_dir.exists():
        shutil.rmtree(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    overrides = [
        "mode=train",
        "data=lm1b-wrap",
        "data.cache_dir=/content/bd3lms/data",
        "data.streaming=true",
        f"data.max_train_samples={max_train_samples}",  # MORE DATA!
        "model=tiny",
        "model.length=128",
        "model.attn_backend=sdpa",
        f"algo={algo}",
        "trainer.accelerator=gpu",
        "trainer.devices=1",
        "trainer.num_nodes=1",
        "trainer.precision=16-mixed",
        "trainer.num_sanity_val_steps=0",
        "trainer.log_every_n_steps=20",
        "trainer.val_check_interval=100",  # Less frequent validation
        f"trainer.max_steps={max_steps}",  # MORE STEPS!
        "data.max_valid_samples=100",
        "data.max_test_samples=100",
        f"checkpointing.save_dir=/content/repro_runs_v2/{run_name}",
        "checkpointing.resume_from_ckpt=false",
        "wandb=null",
    ]
    overrides.extend(_small_loader_overrides(batch_size=8, num_workers=2))

    if block_size is not None:
        overrides.append(f"block_size={block_size}")
    if from_pretrained is not None:
        overrides.append(f"training.from_pretrained={from_pretrained}")
    if extra_overrides:
        overrides.extend(extra_overrides)

    _ = run_main(overrides)
    ckpt = save_dir / "checkpoints" / "last.ckpt"
    if not ckpt.exists():
        raise FileNotFoundError(f"Expected checkpoint not found: {ckpt}")
    return str(ckpt)

def eval_run(algo, checkpoint_path, block_size=None, extra_overrides=None):
    """Evaluate perplexity."""
    overrides = [
        "mode=ppl_eval",
        "data=lm1b-wrap",
        "data.cache_dir=/content/bd3lms/data",
        "data.streaming=true",
        "data.max_test_samples=1000",
        "model=tiny",
        "model.length=128",
        "model.attn_backend=sdpa",
        f"algo={algo}",
        f"eval.checkpoint_path={checkpoint_path}",
        "trainer.accelerator=gpu",
        "trainer.devices=1",
        "trainer.num_nodes=1",
        "trainer.precision=16-mixed",
        "trainer.num_sanity_val_steps=0",
        "wandb=null",
    ]
    overrides.extend(_small_loader_overrides(batch_size=8, num_workers=2))

    if block_size is not None:
        overrides.append(f"block_size={block_size}")
    if extra_overrides:
        overrides.extend(extra_overrides)

    log_text = run_main(overrides)
    ppl = extract_val_ppl(log_text)
    if ppl is None:
        raise ValueError("Could not parse val/ppl from output.")
    return ppl

print("✓ Helper functions loaded!")
print("\nV2 Settings:")
print("  - max_steps: 3000 (was 1500)")
print("  - max_train_samples: 10000 (was 3000)")
print("  - algo.var_min: true (was false)")

✓ Helper functions loaded!

V2 Settings:
  - max_steps: 3000 (was 1500)
  - max_train_samples: 10000 (was 3000)
  - algo.var_min: true (was false)


## Step 1: Check for Existing Base Checkpoint

Χρησιμοποιούμε το υπάρχον base checkpoint αν υπάρχει, αλλιώς κάνουμε train νέο.

In [14]:
# Cell 4: Get or Train Base Checkpoint
from pathlib import Path

# Cell 4: Train Base Checkpoint (Fresh)
print("Training new base checkpoint...")
print("=" * 60)

bd3lm_base_ckpt = train_run_v2(
    "bd3lm_base_len128_v3",  # Νέο όνομα για να μην χρησιμοποιήσει το παλιό
    algo="bd3lm",
    block_size=128,
    max_steps=20000,
    max_train_samples=40000,
    extra_overrides=[
        "training.resample=false",
        "algo.var_min=false",
    ],
)
print(f"✓ Trained new base checkpoint: {bd3lm_base_ckpt}")


Training new base checkpoint...

$ /usr/bin/python3 -u bd3lms/main.py mode=train data=lm1b-wrap data.cache_dir=/content/bd3lms/data data.streaming=true data.max_train_samples=40000 model=tiny model.length=128 model.attn_backend=sdpa algo=bd3lm trainer.accelerator=gpu trainer.devices=1 trainer.num_nodes=1 trainer.precision=16-mixed trainer.num_sanity_val_steps=0 trainer.log_every_n_steps=20 trainer.val_check_interval=100 trainer.max_steps=20000 data.max_valid_samples=100 data.max_test_samples=100 checkpointing.save_dir=/content/repro_runs_v2/bd3lm_base_len128_v3 checkpointing.resume_from_ckpt=false wandb=null loader.global_batch_size=8 loader.eval_global_batch_size=8 loader.batch_size=8 loader.eval_batch_size=8 loader.num_workers=2 trainer.accumulate_grad_batches=1 block_size=128 training.resample=false algo.var_min=false
2.07it/s]

                                                                      
Epoch 16:  86%|████████▌ | 1000/1166 [01:18<00:12, 12.81it/s, v_num=0]Epoch 16, globa

## Step 2: Table 8 Experiments with MORE Training

**Νέες Ρυθμίσεις:**
- `max_steps=3000` (2x από πριν)
- `max_train_samples=10000` (3x από πριν)
- `algo.var_min=true` (ενεργοποιεί variance minimization)

In [15]:
# Cell 5: Table 8 Experiments - V2 with MORE TRAINING

# Define noise schedules
noise_schedules = [
    ("Linear U[0,1]", [
        "training.sampling_eps_min=0.001",
        "training.sampling_eps_max=1.0",
    ]),
    ("Clipped U[0.3,0.8]", [
        "training.sampling_eps_min=0.3",
        "training.sampling_eps_max=0.8",
    ]),
    ("Clipped U[0.45,0.95]", [
        "training.sampling_eps_min=0.45",
        "training.sampling_eps_max=0.95",
    ]),
]

# Only L'=4 for now (faster testing)
Lprime = 4

results_v2 = []

print("=" * 60)
print(f"TABLE 8 v2: MORE TRAINING (L' = {Lprime})")
print("Settings: max_steps=3000, max_train_samples=10000, var_min=true")
print("=" * 60)

for schedule_name, schedule_overrides in noise_schedules:
    print(f"\n{'='*60}")
    print(f"--- {schedule_name} ---")
    print(f"{'='*60}")

    # Create safe run name
    safe_name = schedule_name.replace("[", "").replace("]", "").replace(",", "_").replace(" ", "_")
    run_name = f"bd3lm_v2_{safe_name}_Lp{Lprime}"

    # Fine-tune with MORE STEPS and var_min=true
    finetune_ckpt = train_run_v2(
        run_name,
        algo="bd3lm",
        block_size=Lprime,
        from_pretrained=bd3lm_base_ckpt,
        max_steps=11000,        # 2x MORE!
        max_train_samples=25000,  # 3x MORE!
        extra_overrides=[
            "training.resample=true",
            "algo.var_min=true",  # ENABLED!
        ] + schedule_overrides,
    )

    # Evaluate
    ppl = eval_run(
        algo="bd3lm",
        checkpoint_path=finetune_ckpt,
        block_size=Lprime,
        extra_overrides=["algo.var_min=false"],
    )

    results_v2.append({
        "schedule": schedule_name,
        "ppl": ppl,
    })
    print(f"\n✓ {schedule_name}: PPL = {ppl:.2f}")

print("\n" + "=" * 60)
print("ALL EXPERIMENTS COMPLETE!")
print("=" * 60)

TABLE 8 v2: MORE TRAINING (L' = 4)
Settings: max_steps=3000, max_train_samples=10000, var_min=true

--- Linear U[0,1] ---

$ /usr/bin/python3 -u bd3lms/main.py mode=train data=lm1b-wrap data.cache_dir=/content/bd3lms/data data.streaming=true data.max_train_samples=25000 model=tiny model.length=128 model.attn_backend=sdpa algo=bd3lm trainer.accelerator=gpu trainer.devices=1 trainer.num_nodes=1 trainer.precision=16-mixed trainer.num_sanity_val_steps=0 trainer.log_every_n_steps=20 trainer.val_check_interval=100 trainer.max_steps=11000 data.max_valid_samples=100 data.max_test_samples=100 checkpointing.save_dir=/content/repro_runs_v2/bd3lm_v2_Linear_U0_1_Lp4 checkpointing.resume_from_ckpt=false wandb=null loader.global_batch_size=8 loader.eval_global_batch_size=8 loader.batch_size=8 loader.eval_batch_size=8 loader.num_workers=2 trainer.accumulate_grad_batches=1 block_size=4 training.from_pretrained=/content/repro_runs_v2/bd3lm_base_len128_v3/checkpoints/last.ckpt training.resample=true algo

In [16]:
# Cell 6: Compare Results

print("\n" + "=" * 70)
print("RESULTS COMPARISON: V1 vs V2")
print("=" * 70)

# V1 results (from previous run)
v1_results = {
    "Linear U[0,1]": 1301.36,
    "Clipped U[0.3,0.8]": 1619.97,
    "Clipped U[0.45,0.95]": 1805.48,
}

# Paper results
paper_results = {
    "Linear U[0,1]": 30.18,
    "Clipped U[0.3,0.8]": 29.38,
    "Clipped U[0.45,0.95]": 29.21,
}

print("\n| Schedule | Paper | V1 (1500 steps) | V2 (3000 steps) |")
print("|" + "-"*20 + "|" + "-"*8 + "|" + "-"*17 + "|" + "-"*17 + "|")

for r in results_v2:
    schedule = r['schedule']
    v2_ppl = r['ppl']
    v1_ppl = v1_results.get(schedule, 'N/A')
    paper_ppl = paper_results.get(schedule, 'N/A')
    print(f"| {schedule:<18} | {paper_ppl:<6} | {v1_ppl:<15.2f} | {v2_ppl:<15.2f} |")

# Sort by PPL
print("\n" + "=" * 70)
print("V2 RANKING (sorted by PPL, lower is better):")
print("=" * 70)
for i, r in enumerate(sorted(results_v2, key=lambda x: x['ppl']), 1):
    medal = ["🥇", "🥈", "🥉"][i-1] if i <= 3 else "  "
    print(f"{medal} #{i}: {r['schedule']:<25} PPL = {r['ppl']:.2f}")

# Check if order matches paper
print("\n" + "=" * 70)
print("PAPER EXPECTED ORDER (L'=4):")
print("  #1: Clipped U[0.45,0.95] (Best - heavy masking)")
print("  #2: Clipped U[0.3,0.8]")
print("  #3: Linear U[0,1] (Worst)")
print("=" * 70)

v2_sorted = sorted(results_v2, key=lambda x: x['ppl'])
if "0.45" in v2_sorted[0]['schedule']:
    print("\n✅ SUCCESS! Order matches paper!")
elif v2_sorted[0]['ppl'] < v1_results.get(v2_sorted[0]['schedule'], float('inf')):
    print("\n⚠️ Order doesn't match paper, but PPL improved!")
else:
    print("\n❌ Order still doesn't match - may need even more training")


RESULTS COMPARISON: V1 vs V2

| Schedule | Paper | V1 (1500 steps) | V2 (3000 steps) |
|--------------------|--------|-----------------|-----------------|
| Linear U[0,1]      | 30.18  | 1301.36         | 257.00          |
| Clipped U[0.3,0.8] | 29.38  | 1619.97         | 258.92          |
| Clipped U[0.45,0.95] | 29.21  | 1805.48         | 260.57          |

V2 RANKING (sorted by PPL, lower is better):
🥇 #1: Linear U[0,1]             PPL = 257.00
🥈 #2: Clipped U[0.3,0.8]        PPL = 258.92
🥉 #3: Clipped U[0.45,0.95]      PPL = 260.57

PAPER EXPECTED ORDER (L'=4):
  #1: Clipped U[0.45,0.95] (Best - heavy masking)
  #2: Clipped U[0.3,0.8]
  #3: Linear U[0,1] (Worst)

⚠️ Order doesn't match paper, but PPL improved!


## Optional: Run L'=16 if V2 shows improvement

In [17]:
# Cell 7 (Optional): Run L'=16

RUN_L16 = True  # Set to True to run L'=16 experiments

if RUN_L16:
    Lprime = 16
    results_v2_L16 = []

    print("\n" + "=" * 60)
    print(f"TABLE 8 v2: L' = {Lprime}")
    print("=" * 60)

    for schedule_name, schedule_overrides in noise_schedules:
        print(f"\n--- {schedule_name} ---")

        safe_name = schedule_name.replace("[", "").replace("]", "").replace(",", "_").replace(" ", "_")
        run_name = f"bd3lm_v2_{safe_name}_Lp{Lprime}"

        finetune_ckpt = train_run_v2(
            run_name,
            algo="bd3lm",
            block_size=Lprime,
            from_pretrained=bd3lm_base_ckpt,
            max_steps=5000,
            max_train_samples=20000,
            extra_overrides=[
                "training.resample=true",
                "algo.var_min=true",
            ] + schedule_overrides,
        )

        ppl = eval_run(
            algo="bd3lm",
            checkpoint_path=finetune_ckpt,
            block_size=Lprime,
            extra_overrides=["algo.var_min=false"],
        )

        results_v2_L16.append({"schedule": schedule_name, "ppl": ppl})
        print(f"✓ {schedule_name}: PPL = {ppl:.2f}")

    print("\nL'=16 Results:")
    for r in sorted(results_v2_L16, key=lambda x: x['ppl']):
        print(f"  {r['schedule']}: {r['ppl']:.2f}")
else:
    print("Skipping L'=16 experiments. Set RUN_L16=True to run.")


TABLE 8 v2: L' = 16

--- Linear U[0,1] ---

$ /usr/bin/python3 -u bd3lms/main.py mode=train data=lm1b-wrap data.cache_dir=/content/bd3lms/data data.streaming=true data.max_train_samples=20000 model=tiny model.length=128 model.attn_backend=sdpa algo=bd3lm trainer.accelerator=gpu trainer.devices=1 trainer.num_nodes=1 trainer.precision=16-mixed trainer.num_sanity_val_steps=0 trainer.log_every_n_steps=20 trainer.val_check_interval=100 trainer.max_steps=5000 data.max_valid_samples=100 data.max_test_samples=100 checkpointing.save_dir=/content/repro_runs_v2/bd3lm_v2_Linear_U0_1_Lp16 checkpointing.resume_from_ckpt=false wandb=null loader.global_batch_size=8 loader.eval_global_batch_size=8 loader.batch_size=8 loader.eval_batch_size=8 loader.num_workers=2 trainer.accumulate_grad_batches=1 block_size=16 training.from_pretrained=/content/repro_runs_v2/bd3lm_base_len128_v3/checkpoints/last.ckpt training.resample=true algo.var_min=true training.sampling_eps_min=0.001 training.sampling_eps_max=1.0
%

KeyboardInterrupt: 

## Summary

**Αν η διάταξη ακόμα δεν ταιριάζει:**

Μπορείς να δοκιμάσεις:
1. `max_steps=5000` (ακόμα περισσότερο training)
2. `max_train_samples=20000`
3. Larger batch size αν έχεις αρκετή VRAM

**Για το report:**
> Με scaled-down training, η διάταξη μπορεί να διαφέρει από το paper λόγω του ότι τα clipped schedules
> χρειάζονται περισσότερο training για να δείξουν το πλεονέκτημά τους στη μείωση της variance.